In [ ]:
#FastAPI Assignment

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from motor.motor_asyncio import AsyncIOMotorClient

app = FastAPI()

# MongoDB connection
client = AsyncIOMotorClient("mongodb://localhost:27017/")
db = client["mydatabase"]
items_collection = db["items"]
clock_in_records_collection = db["clock_in_records"]

# Pydantic models
class Item(BaseModel):
    name: str
    email: str
    item_name: str
    quantity: int
    expiry_date: str
    insert_date: str = None

class ClockInRecord(BaseModel):
    email: str
    location: str
    insert_date_time: str = None

# Items API
@app.post("/items")
async def create_item(item: Item):
    item.insert_date = datetime.now().strftime("%Y-%m-%d")
    result = await items_collection.insert_one(item.dict())
    return {"id": result.inserted_id}

@app.get("/items/{id}")
async def get_item(id: str):
    result = await items_collection.find_one({"_id": ObjectId(id)})
    if result:
        return result
    else:
        raise HTTPException(status_code=404, detail="Item not found")

@app.get("/items/filter")
async def filter_items(email: str = None, expiry_date: str = None, insert_date: str = None, quantity: int = None):
    filters = {}
    if email:
        filters["email"] = email
    if expiry_date:
        filters["expiry_date"] = {"$gte": expiry_date}
    if insert_date:
        filters["insert_date"] = {"$gte": insert_date}
    if quantity:
        filters["quantity"] = {"$gte": quantity}
    results = await items_collection.find(filters).to_list(100)
    return results

@app.delete("/items/{id}")
async def delete_item(id: str):
    result = await items_collection.delete_one({"_id": ObjectId(id)})
    if result.deleted_count == 1:
        return {"message": "Item deleted"}
    else:
        raise HTTPException(status_code=404, detail="Item not found")

@app.put("/items/{id}")
async def update_item(id: str, item: Item):
    result = await items_collection.update_one({"_id": ObjectId(id)}, {"$set": item.dict(exclude={"insert_date"})})
    if result.modified_count == 1:
        return {"message": "Item updated"}
    else:
        raise HTTPException(status_code=404, detail="Item not found")

@app.get("/items/aggregation")
async def aggregate_items():
    pipeline = [{"$group": {"_id": "$email", "count": {"$sum": 1}}}]
    results = await items_collection.aggregate(pipeline).to_list(100)
    return results

# Clock-In Records API
@app.post("/clock-in")
async def create_clock_in_record(clock_in_record: ClockInRecord):
    clock_in_record.insert_date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    result = await clock_in_records_collection.insert_one(clock_in_record.dict())
    return {"id": result.inserted_id}

@app.get("/clock-in/{id}")
async def get_clock_in_record(id: str):
    result = await clock_in_records_collection.find_one({"_id": ObjectId(id)})
    if result:
        return result
    else:
        raise HTTPException(status_code=404, detail="Clock-in record not found")

@app.get("/clock-in/filter")
async def filter_clock_in_records(email: str = None, location: str = None, insert_date_time: str = None):
    filters = {}
    if email:
        filters["email"] = email
    if location:
        filters["location"] = location
    if insert_date_time:
        filters["insert_date_time"] = {"$gte": insert_date_time}
    results = await clock_in_records_collection.find(filters).to_list(100)
    return results

@app.delete("/clock-in/{id}")
async def delete_clock_in_record(id: str):
    result = await clock_in_records_collection.delete_one({"_id": ObjectId(id)})
    if result.deleted_count == 1:
        return {"message": "Clock-in record deleted"}
    else:
        raise HTTPException(status_code=404, detail="Clock-in record not found

@app.put("/clock-in/{id}")
async def update_clock_in_record(id: str, clock_in_record: ClockInRecord):
    result = await clock_in_records_collection.update_one({"_id": ObjectId(id)}, {"$set": clock_in_record.dict(exclude={"insert_date_time"})})
    if result.modified_count == 1:
        return {"message": "Clock-in record updated"}
    else:
        raise HTTPException(status_code=404, detail="Clock-in record not found")                           